In [2]:
pip install psycopg2


Note: you may need to restart the kernel to use updated packages.


In [ ]:
import psycopg2
import pandas as pd


# 数据库连接信息
host = "localhost"
dbname = "THSR_database"
user = "postgres"
password = "1234"

# 连接到 PostgreSQL 数据库
conn = psycopg2.connect(
    host=host,
    dbname=dbname,
    user=user,
    password=password
)

# 创建一个游标（cursor）
cur = conn.cursor()

# 执行 SQL 查询
cur.execute("SELECT arrive_station_id, citizen_id FROM reserved_ticket WHERE travel_date = '2023-08-01'")

# 获取查询结果
rows = cur.fetchall()

# 打印每一行
# for row in rows:
#     print(row)

# 获取查询结果并转换为 pandas DataFrame
df = pd.DataFrame(rows, columns=["arrive_station_id", "citizen_id"])

# 显示 DataFrame
print(df)

# 关闭游标和连接
cur.close()
conn.close()


       arrive_station_id citizen_id
0                    990  AAAAW1776
1                   1010  AAABC5393
2                   1043  ABBAI4412
3                   1000  AAAAG9104
4                   1070  AAAAI6243
...                  ...        ...
93665               1020  AAAAZ2053
93666               1060  AAAAZ8454
93667               1050  AAABC7075
93668               1060  AAAAG3584
93669               1030  AAAAJ5984

[93670 rows x 2 columns]


# Hash function

In [20]:
import psycopg2
import pandas as pd
import time

host = "localhost"
dbname = "THSR_database"
user = "postgres"
password = "1234"

conn = psycopg2.connect(
    host=host,
    dbname=dbname,
    user=user,
    password=password
)

cur = conn.cursor()

# Step 1
cur.execute("""
    SELECT arrive_station_id, citizen_id
    FROM reserved_ticket
    WHERE travel_date = '2023-08-01' AND depart_station_id <= 1000
""")
reserved_ticket_rows = cur.fetchall()

reserved_ticket_df = pd.DataFrame(reserved_ticket_rows, columns=["arrive_station_id", "citizen_id"])
print(f"Reserved Ticket records: {len(reserved_ticket_df)}")

# Step 2：Hash function
def hash_function(citizen_id):
    return sum(ord(c) for c in citizen_id) % 100

buckets = {i: [] for i in range(100)}

# Put reserved_ticket data into buckets based on the hash function
for index, row in reserved_ticket_df.iterrows():
    citizen_id = row["citizen_id"]
    bucket_index = hash_function(citizen_id)
    buckets[bucket_index].append({"arrive_station_id": row["arrive_station_id"], "citizen_id": citizen_id, "name": None})


# Step 3
cur.execute("""
    SELECT citizen_id, name
    FROM member
    WHERE name LIKE '% A%'
""")
member_rows = cur.fetchall()

member_df = pd.DataFrame(member_rows, columns=["citizen_id", "name"])
print(f"Member records: {len(member_df)}")

# Step 4: Hash and join member data
start_hash_join = time.time()
for index, row in member_df.iterrows():
    citizen_id = row["citizen_id"]
    name = row["name"]
    bucket_index = hash_function(citizen_id)

    # Search for matching reserved_ticket data
    for reserved_ticket in buckets[bucket_index]:
        if reserved_ticket["citizen_id"] == citizen_id:
            reserved_ticket["name"] = name  # Fill the name column
end_hash_join = time.time()

# Step 5
result = []
for bucket in buckets.values():
    for reserved_ticket in bucket:
        if reserved_ticket["name"] is not None:  # Only output records with a filled name
            result.append((reserved_ticket["arrive_station_id"], reserved_ticket["name"]))

result_df = pd.DataFrame(result, columns=["arrive_station_id", "name"])

print(f"Hash join execution time: {end_hash_join - start_hash_join:.6f} seconds")
print(result_df)

cur.close()
conn.close()


Reserved Ticket records: 21290
Member records: 10248
Hash join execution time: 0.568619 seconds
     arrive_station_id                   name
0                  990          Michael Arvin
1                 1043         Brenda Aguilar
2                 1047  Christopher Ardizzone
3                  990                Van Aki
4                 1010         Andrew Aguilar
..                 ...                    ...
522               1070          Elaine Arthur
523               1020            Julie Arias
524                990         Tanya Anderson
525               1047       Jessica Anderson
526               1040       Charles Anderson

[527 rows x 2 columns]


In [18]:
count_60 = len(buckets[60])
count_70 = len(buckets[70])
count_80 = len(buckets[80])

print(f"Bucket 60 contains {count_60} records.")
print(f"Bucket 70 contains {count_70} records.")
print(f"Bucket 80 contains {count_80} records.")

Bucket 60 contains 291 records.
Bucket 70 contains 27 records.
Bucket 80 contains 0 records.


# Brute Force

In [ ]:
import psycopg2
import pandas as pd
import time

host = "localhost"
dbname = "THSR_database"
user = "postgres"
password = "1234"

conn = psycopg2.connect(
    host=host,
    dbname=dbname,
    user=user,
    password=password
)

cur = conn.cursor()

# Step 1
cur.execute("""
    SELECT arrive_station_id, citizen_id
    FROM reserved_ticket
    WHERE travel_date = '2023-08-01' AND depart_station_id <= 1000
""")
reserved_ticket_rows = cur.fetchall()

reserved_ticket_df = pd.DataFrame(reserved_ticket_rows, columns=["arrive_station_id", "citizen_id"])

# Step 3
cur.execute("""
    SELECT citizen_id, name
    FROM member
    WHERE name LIKE '% A%'
""")
member_rows = cur.fetchall()

member_df = pd.DataFrame(member_rows, columns=["citizen_id", "name"])

# Step 4: Brute-force join using nested loops
result = []

reserved_ticket_array = reserved_ticket_df.to_numpy()
member_array = member_df.to_numpy()

# Nested loops to perform the join
start_time = time.time()
for member in member_array:
    matches = reserved_ticket_array[:, 1] == member[0]  # Compare citizen_id
    for idx, match in enumerate(matches):
        if match:
            result.append((reserved_ticket_array[idx, 0], member[1]))  # arrive_station_id, name

brute_force_time = time.time() - start_time

# Step 5
result_df = pd.DataFrame(result, columns=["arrive_station_id", "name"])

print(f"Brute-force join execution time: {brute_force_time:.6f} seconds")
print(result_df)

cur.close()
conn.close()

Brute-force join execution time: 15.090085 seconds
     arrive_station_id              name
0                 1035      James Acosta
1                 1010      Sherri Ahner
2                  990       Alice Adams
3                 1040      Debbie Allen
4                  990     Michael Arvin
..                 ...               ...
522                990      Lisa Atchley
523               1020      Margit Adams
524                990       Andy Andrew
525               1010  Connie Alejandro
526                990      Cathy Arroyo

[527 rows x 2 columns]


In [34]:
import math
M = 5000
N = 40000
m = 500000
n = 4000000
B = 100

stupid = M+m*N
Single_block = M+M*N
ceil = M/(B-2)
ceil = math.ceil(ceil)
Mutile_block = M+ceil*N
t = 0.001

print(stupid)
print(Single_block)
print(Mutile_block)

print(stupid*t)
print(Single_block*t)
print(Mutile_block*t)

20000005000
200005000
2085000
20000005.0
200005.0
2085.0
